In [179]:
# pip unistall ipywidgets
# pip install ipywidgets
# pip install tabulate
# pip install plotly 

In [180]:
import ipywidgets as w
from IPython.display import display, clear_output
import json

from utils import *

with open('data.txt', 'r') as f:
    parts = json.load(f)

In [ ]:
def combinations(parts, k):
    if k > 1:
        comb = [(*x, y) for x in combinations(parts, k-1) for y in parts[k]]
        return comb
    else:
        comb = [(x, y) for x in parts[k-1] for y in parts[k]]
        return comb
    
def get_attribute_sum(comb, attr):
    attr_sum = 0
    for part in comb:
        attr_sum += list(part.values())[0][attr]
    return attr_sum

In [186]:
### INITIAL FORM | VISUALIZATION ###

####################################################################################################
# Іміджева складова
####################################################################################################
w1_widgets = [
    widget_typeBox('Материал корпуса', get_part_attr(parts, 'Body', 'material')),
    widget_flag('Портативность')
]
w1 = widget_group('Внешний вид', w1_widgets)

####################################################################################################
# Механічна складова
####################################################################################################
w2_widgets = [
    widget_typeBox('Тип двигателя', get_part_attr(parts, 'Engine', 'engine_type')),
    widget_rangeBoxInt('Число двигателей', *get_boundaries(parts, 'Engine', 'number')),
    widget_typeBox('Тип движителя', get_part_attr(parts, 'Engine', 'propulsion_type')),
    
    widget_typeBox('Материал пропеллера', get_part_attr(parts, 'Propeller', 'material')),
    widget_rangeBoxInt('Число пропеллеров', *get_boundaries(parts, 'Propeller', 'number')),
]
w2 = widget_group('Механика', w2_widgets)

####################################################################################################
# Апаратна складова
####################################################################################################
w3_widgets = [
    widget_flag('Наличие GPS'),
    widget_flag('Наличие ИИ'),
    widget_typeBox('Производитель ', get_part_attr(parts, 'Software', 'manufacturer')),
    
    widget_rangeBoxFloat('Частота процессора, Гц', *get_boundaries(parts, 'CPU', 'frequency')),
    widget_rangeBoxInt('Количество ядер процессора', *get_boundaries(parts, 'CPU', 'num_cores')),
    
    widget_rangeBoxFloat('Дальность приема сигнала, м', *get_boundaries(parts, 'Sensor',
                                                                        'signal_acceptance_distance')),
    
    widget_rangeBoxFloat('Разрешение матрицы, Мп', *get_boundaries(parts, 'Camera', 'resolution')),
    widget_rangeBoxInt('Угол обзора', *get_boundaries(parts, 'Camera', 'angle')),
    widget_rangeBoxInt('Дальность ИК подсветки, м', *get_boundaries(parts, 'Camera', 'IR_illumination')),
]
w3 = widget_group('Аппаратная составляющая', w3_widgets)

####################################################################################################
# Акумулятор
####################################################################################################
w4_widgets = [
    widget_rangeBoxInt('Продолжительность полета, мин', *get_boundaries(parts, 'Accumulator', 'duration')),
    widget_rangeBoxInt('Взлетная масса, кг', *get_boundaries(parts, 'Accumulator', 'take_off_weight')),
    widget_rangeBoxInt('Высота полета, м', *get_boundaries(parts, 'Accumulator', 'height')),
]
w4 = widget_group('Аккумулятор', w4_widgets)

####################################################################################################
# Total
####################################################################################################
w5_widgets = [
#     widget_rangeBox('Общая стоимость', 0, 1000),
#     widget_rangeBox('Общий вес', 0, 1000)
     widget_rangeBoxInt('Стоимость, грн', 0, 30000),
     widget_rangeBoxInt('Общий вес, г', 0, 1000),
]
layout = w.Layout(width='480px', border='solid 0px', margin='10px',
                  justify_content='center', align_items='center')
w5 = widget_group('', w5_widgets, layout=layout)

In [309]:
def analyse(b, parts, window):
    clear_output()
    tab.active = 1
    display(tab)
#     display(window)
    restriction = {}
    for key in get_keys(parts):
        restriction[key] = {}
        
    restriction['Body']['material'] = w1.children[1].value
    restriction['Body']['portability'] = w1.children[2].value

    restriction['Engine']['engine_type'] = w2.children[1].value
    restriction['Engine']['number'] = w2.children[2].value
    restriction['Engine']['propulsion_type'] = w2.children[3].value
    restriction['Propeller']['material'] = w2.children[4].value
    restriction['Propeller']['number'] = w2.children[5].value

    restriction['Software']['GPS'] = w3.children[1].value
    restriction['Software']['AI'] = w3.children[2].value
    restriction['Software']['manufacturer'] = w3.children[3].value
    restriction['CPU']['frequency'] = w3.children[4].value
    restriction['CPU']['num_cores'] = w3.children[5].value
    restriction['Sensor']['signal_acceptance_distance'] = w3.children[6].value
    restriction['Camera']['resolution'] = w3.children[7].value
    restriction['Camera']['angle'] = w3.children[8].value
    restriction['Camera']['IR_illumination'] = w3.children[9].value

    restriction['Accumulator']['duration'] = w4.children[1].value
    restriction['Accumulator']['take_off_weight'] = w4.children[2].value
    restriction['Accumulator']['height'] = w4.children[3].value
    
    restriction['Total'] = {}
    restriction['Total']['cost'] = w5.children[1].value
    restriction['Total']['weight'] = w5.children[2].value
    
#     print(restriction)
    brute_force(b, parts, restriction)

In [310]:
def brute_force(b, parts, restriction):
    new_parts = []
    for part in get_keys(parts): #restriction.keys():
        particular_parts = filter_parts(parts, part)
        for example in particular_parts:
            example_satisfies_conditions = True
            for attr in restriction[part].keys():
                if isinstance(restriction[part][attr], str):
                    condition = example[part][attr] == restriction[part][attr]
                elif isinstance(restriction[part][attr], tuple):
                    min_value, max_value = restriction[part][attr]
                    condition = example[part][attr] >= min_value and example[part][attr] <= max_value
                elif isinstance(restriction[part][attr], bool):
                    condition = not restriction[part][attr] or (example[part][attr] 
                                                                and restriction[part][attr])
                if not condition:
                    example_satisfies_conditions = False
                    break
            if example_satisfies_conditions:
                new_parts.append(example)

#     print(list(map(lambda part: '{}: {}'.format(part, filter_parts(new_parts, part) != []), get_keys(parts))))
    if not all(filter_parts(new_parts, part) != [] for part in get_keys(parts)):
        print('Bad news :(')
        return
        
    new_pparts = []
    for part in get_keys(new_parts):
        new_pparts.append(filter_parts(new_parts, part))
    filtered_combinations = []
    for comb in combinations(new_pparts, len(pparts)-1):
        min_cost, max_cost = restriction['Total']['cost']
        min_weight, max_weight = restriction['Total']['weight']
        comb_cost = get_attribute_sum(comb, 'price')
        comb_weight = get_attribute_sum(comb, 'weight')
        if comb_cost >= min_cost and comb_cost <= max_cost and \
        comb_weight >= min_weight and comb_weight <= max_weight:
            filtered_combinations.append(comb)
    if len(filtered_combinations) == 0:
        print('Bad news :(')
        return
    visualize_structure(b, filtered_combinations, 1)

In [311]:
def visualize_structure(b, filtered_combinations, i):
    print('Множество Паретто')
    print('Мощность множества = {}'.format(len(filtered_combinations)))
    print('Выбранная структура: {}\n'.format(i))
    for part in filtered_combinations[i-1]:
        print('{} {}'.format(list(part.keys())[0], list(part.values())[0]['name']))
        for key in set(list(part.values())[0].keys()) - {'name', 'price', 'weight'}:
            print('\t> {}: {}'.format(key, list(part.values())[0][key]))
        print()
    

In [312]:
b1 = w.Button(
    description='Анализ',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

b1.on_click(lambda b: analyse(b, parts, window))

In [316]:
all_widgets = w.HBox([w.VBox([w1, w2, w5, b1], layout=bigGroup), w.VBox([w3, w4], layout=bigGroup)])

window = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
                widget_header('Дрон. Требования к функциональным элементам', bold=False, size=5, face=facefont),
                all_widgets], layout=allLayout) 

widgets_structure = w.Label('Множество Паретто')
widgets_priority = w.Label('Приоритетное требование')

tab_contents = ['Требования', 'Выбор структуры', 'Приоритетное требование']
children = [window, widgets_structure, widgets_priority]
tab = w.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])
display(tab)

Tab(children=(VBox(children=(HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Lab 5</font></div>"), HTML(value="<div align='center'><font size='5px' face='Arial' color='#0969A2'>Дрон. Требования к функциональным элементам</font></div>"), HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Внешний вид</font></div>"), Dropdown(description='Материал корпуса', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Кевлар', 'Углепластика', 'Экструдированный пенополистирол'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), Checkbox(value=False, description='Портативность', style=DescriptionStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механика</font></div>"), Dropdown(description='Тип двигателя', layout=Layout(justify_content='center', width='400px'), options=('Бесколлекторный электродвигатель', 'Коллекторный электродвигатель', 'Электродвигатель'), style=DescriptionStyle(description_width='initial'), value='Бесколлекторный электродвигатель'), FloatRangeSlider(value=(2.0, 6.0), continuous_update=False, description='Число двигателей', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), Dropdown(description='Тип движителя', layout=Layout(justify_content='center', width='400px'), options=('Винт', 'Реактивная тяга'), style=DescriptionStyle(description_width='initial'), value='Винт'), Dropdown(description='Материал пропеллера', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Карбон'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), FloatRangeSlider(value=(4.0, 6.0), continuous_update=False, description='Число пропеллеров', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), FloatRangeSlider(value=(0.0, 21797.0), continuous_update=False, description='Стоимость, грн', layout=Layout(justify_content='flex-start', width='400px'), max=30000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), FloatRangeSlider(value=(0.0, 1000.0), continuous_update=False, description='Общий вес, г', layout=Layout(justify_content='flex-start', width='400px'), max=1000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='10px', width='480px')), Button(description='Анализ', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Аппаратная составляющая</font></div>"), Checkbox(value=False, description='Наличие GPS', style=DescriptionStyle(description_width='initial')), Checkbox(value=False, description='Наличие ИИ', style=DescriptionStyle(description_width='initial')), Dropdown(description='Производитель ', layout=Layout(justify_content='center', width='400px'), options=('Agisoft', 'Harris Geospatial Solutions', 'Trimble'), style=DescriptionStyle(description_width='initial'), value='Agisoft'), FloatRangeSlider(value=(2.1, 3.0), continuous_update=False, description='Частота процессора, Гц', layout=Layout(justify_content='flex-start', width='400px'), ma

Множество Паретто
Мощность множества = 8
Выбранная структура: 1

Camera AVT AHDCH 108
	> IR_illumination: 18
	> angle: 180
	> resolution: 2

Engine Syma NHYDXS
	> engine_type: Бесколлекторный электродвигатель
	> number: 2
	> propulsion_type: Винт

Sensor VDI/VDE 3845
	> signal_acceptance_distance: 2

Accumulator Skywalker X8
	> take_off_weight: 8
	> duration: 240
	> height: 1200

CPU DSP C66x
	> frequency: 2.7
	> num_cores: 8

Body AgDrone
	> portability: True
	> material: Волоконный композит

Software Agisoft Photoscan
	> manufacturer: Agisoft
	> GPS: True
	> AI: False

Propeller Phantom 5
	> number: 4
	> material: Волоконный композит



### Ответ

In [132]:
####################################################################################################
# Іміджева складова
####################################################################################################
rw1_widgets = [
    widget_typeBox('Корпус', get_part_attr(parts, 'Body', 'name')),
]

rw1 = widget_group('Зовнішній вигляд', rw1_widgets)

####################################################################################################
# Механічна складова
####################################################################################################
rw2_widgets = [
    widget_typeBox('Двигуни', get_part_attr(parts, 'Engine', 'name')),
    
    widget_typeBox('Проперели', get_part_attr(parts, 'Propeller', 'name')),
]
rw2 = widget_group('Механіка', rw2_widgets)

####################################################################################################
# Апаратна складова
####################################################################################################
rw3_widgets = [
    widget_typeBox('Процесор', get_part_attr(parts, 'CPU', 'name')),
    
    widget_typeBox('Датчик зв\'язку', get_part_attr(parts, 'Sensor', 'name')),
    
    widget_typeBox('Програмне забезпечення', get_part_attr(parts, 'Software', 'name')),
    widget_typeBox('Камера', get_part_attr(parts, 'Camera', 'name')),
]
rw3 = widget_group('Апаратна складова', rw3_widgets)

####################################################################################################
# Акумулятор
####################################################################################################
rw4_widgets = [
    widget_typeBox('Акумулятор', get_part_attr(parts, 'Accumulator', 'name')),
]
rw4 = widget_group('Акумулятор', rw4_widgets)

####################################################################################################
# Total
####################################################################################################
totalLayout = w.Layout(width='200px', justify_content='flex-end')
rw5_widgets = [
      w.HBox([w.Label(value='Загальна вартість:', style=title_style),
              w.Label(value='', style=title_style)], layout=totalLayout),
      w.HBox([w.Label(value='Загальна вага:', style=title_style),
              w.Label(value='', style=title_style)], layout=totalLayout),
]
rw5 = widget_group('', rw5_widgets)

In [133]:
def get_price(body, engine, propellers, cpu, sensor, software, camera, accumulator):
    names = list(map(lambda part: list(part.values())[0]['name'], parts))
    price = 0
    for name in [body, engine, propellers, cpu, sensor, software, camera, accumulator]:
        assert name in names, '{} is not a valid name'.format(name)
        price += get_attr(list(filter(lambda part: list(part.values())[0]['name'] == name, parts)), 'price')[0]
    return price

def get_weight(body, engine, propellers, cpu, sensor, software, camera, accumulator):
    names = list(map(lambda part: list(part.values())[0]['name'], parts))
    weight = 0
    for name in [body, engine, propellers, cpu, sensor, software, camera, accumulator]:
        assert name in names, '{} is not a valid name'.format(name)
        weight += get_attr(list(filter(lambda part: list(part.values())[0]['name'] == name, parts)), 'weight')[0]
    return weight

def calculate(b):
    rw5.children[1].children[1].value = str(get_price(rw1.children[1].value,
                                                      rw2.children[1].value, rw2.children[2].value,
                                                      rw3.children[1].value, rw3.children[2].value,
                                                      rw3.children[3].value, rw3.children[4].value,
                                                      rw4.children[1].value)) + ' грн'
    rw5.children[2].children[1].value = str(get_weight(rw1.children[1].value,
                                                       rw2.children[1].value, rw2.children[2].value,
                                                       rw3.children[1].value, rw3.children[2].value,
                                                       rw3.children[3].value, rw3.children[4].value,
                                                       rw4.children[1].value)) + ' г'

In [134]:
rb1 = w.Button(
    description='Прорахувати',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

rb1.on_click(calculate)

all_widgets = w.HBox([w.VBox([rw1, rw2, rw5], layout=bigGroup), w.VBox([rw3, rw4, rb1], layout=bigGroup)],
                     layout=allLayout)

display(widget_header('Lab 5'))
display(widget_header('Drone', bold=False))
display(all_widgets)

HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'><b>Lab 5</b></font></div>")

HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Drone</font></div>")

HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Зовнішній вигляд</font></div>"), Dropdown(description='Корпус', layout=Layout(justify_content='center', width='400px'), options=('AgDrone', 'Hubsan H501S', 'Hubsan H551O', 'Hubsan H818J', 'Raytheon', 'Тахион'), style=DescriptionStyle(description_width='initial'), value='AgDrone')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механіка</font></div>"), Dropdown(description='Двигуни', layout=Layout(justify_content='center', width='400px'), options=('DJI SPARK', 'Rotron LAZ29', 'Rotron VVMH7', 'Rotron Y3TKO', 'Syma NHYDXS', 'Syma X5SW\xa0', 'Syma X8HW', 'WL Toys WL-Q282Jw'), style=DescriptionStyle(description_width='initial'), value='DJI SPARK'), Dropdown(description='Проперели', layout=Layout(justify_content='center', width='400px'), options=('AEE N7638', 'AEE Propeller', 'DJI 9450 CW', 'DJI Inspire 2', 'Phantom 1', 'Phantom 3', 'Phantom 5'), style=DescriptionStyle(description_width='initial'), value='AEE N7638')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), HBox(children=(Label(value='Загальна вартість:', style=DescriptionStyle(description_width='initial')), Label(value='', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px')), HBox(children=(Label(value='Загальна вага:', style=DescriptionStyle(description_width='initial')), Label(value='', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px'))), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Апаратна складова</font></div>"), Dropdown(description='Процесор', layout=Layout(justify_content='center', width='400px'), options=('C6678', 'DSP C66x', 'DSP А871', 'TMS320C6678'), style=DescriptionStyle(description_width='initial'), value='C6678'), Dropdown(description="Датчик зв'язку", layout=Layout(justify_content='center', width='400px'), options=('ATEX', 'DIN EN', 'Namur', 'VDI/VDE 3845'), style=DescriptionStyle(description_width='initial'), value='ATEX'), Dropdown(description='Програмне забезпечення', layout=Layout(justify_content='center', width='400px'), options=('Agisoft Photoscan', 'ENVI OneButton', 'PHOTOMOD UAS', 'UASMaster'), style=DescriptionStyle(description_width='initial'), value='Agisoft Photoscan'), Dropdown(description='Камера', layout=Layout(justify_content='center', width='400px'), options=('AVT 3300AUXSD', 'AVT 6600AUXSDWF', 'AVT AHDCH 108', 'JIMI JH012'), style=DescriptionStyle(description_width='initial'), value='AVT 3300AUXSD')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Акумулятор</font></div>"), Dropdown(description='Акумулятор', layout=Layout(justify_content='center', width='400px'), options=('Aladin', 'LiPo', 'Luftera', 'Skywalker X8'), style=DescriptionStyle(description_width='initial'), value='Aladin')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), Button(description='Прорахувати', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px'))), layout=Layout(ali

In [135]:
# get_price(body, engine, propellers, cpu, sensor, software, camera, accumulator)

In [ ]:

parts = [['aa', 'baa', 'caa'], ['dgg', 'fgg'], ['1', '2ss'], ['3', '4']]
combinations(parts, len(parts)-1)

In [145]:
import json
with open('data.txt', 'r') as f:
    parts = json.load(f)

# combinations(parts, len(parts)-1)

pparts = []
for part in get_keys(parts):
    pparts.append(filter_parts(parts, part))

In [146]:
combs = combinations(pparts, len(pparts)-1)

In [158]:
get_attribute_sum(combs[0], 'price')

29560